<a href="https://colab.research.google.com/github/ADRIAN02-CODE/telco-churn-prediction/blob/main/meningioma_detector_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_dir = '/content/drive/MyDrive/DSGP33 (meningioma)'


In [ ]:
data_dir = '/content/drive/MyDrive/DSGP33 (meningioma)'
train_dir = data_dir + '/train'
test_dir = data_dir + '/test'


In [ ]:
train_dir = '/content/drive/MyDrive/DSGP33 (meningioma )/train'
test_dir = '/content/drive/MyDrive/DSGP33 (meningioma )/test'



In [ ]:
import os

print(os.listdir('/content/drive/MyDrive'))
# Try both possibilities!
try:
    print("With space and parentheses:", os.listdir('/content/drive/MyDrive/DSGP33 (meningioma)'))
except Exception as e:
    print("Error with space and parentheses:", e)
try:
    print("Without space:", os.listdir('/content/drive/MyDrive/DSGP33(meningioma)'))
except Exception as e:
    print("Error without space:", e)


['Colab Notebooks', '94 Batch Accounts.gsheet', 'Saved from Chrome', 'Google Earth', 'DSGP33 (meningioma )']
Error with space and parentheses: [Errno 2] No such file or directory: '/content/drive/MyDrive/DSGP33 (meningioma)'
Error without space: [Errno 2] No such file or directory: '/content/drive/MyDrive/DSGP33(meningioma)'


In [ ]:
import tensorflow as tf

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=False
)


Found 5974 files belonging to 2 classes.
Found 1311 files belonging to 2 classes.


In [ ]:
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
    layers.Resizing(224, 224),           # Ensures all images are correct size
    layers.Rescaling(1./255),            # Normalizes pixel values
    layers.RandomFlip('horizontal'),     # Augment only on training
    layers.RandomRotation(0.05),
])

# You can apply `data_augmentation` when building your model, or use `tf.data` map for train_ds only.


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0

NUM_CLASSES = len(train_ds.class_names)

data_augmentation = keras.Sequential([
    layers.Resizing(224, 224),
    layers.Rescaling(1./255),
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.05),
])

base_model = EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)
base_model.trainable = False

inputs = layers.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = models.Model(inputs, outputs)
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         2,562 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,052,133 (15.46 MB)

 Trainable params: 2,562 (10.01 KB)

 Non-trainable params: 4,049,571 (15.45 MB)